In [9]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [11]:
%run -i ../../zkstats/core.py

In [12]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [13]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.tensor(np.median(dummy_data))
# print(int(len(dummy_data)/2))
dummy_lower_to_median = torch.tensor(np.sort(dummy_data)[int(len(dummy_data)/2)-1])
dummy_upper_to_median = torch.tensor(np.sort(dummy_data)[int(len(dummy_data)/2)])


In [14]:
print("dummy output: ", dummy_theory_output)
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = dummy_lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = dummy_upper_to_median, requires_grad = False)
    def forward(self,X):
        # since within 1%, we regard as same value
        count_less = torch.sum((X < 0.99*self.w).double())
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        len = X.size()[1]
        half_len = torch.floor(torch.div(len, 2))
        
        # not support modulo yet
        less_cons = count_less<half_len+2*(len/2 - torch.floor(len/2))
        more_cons = count_less+count_equal>half_len

        # For count_equal == 0
        lower_exist = torch.sum((torch.abs(X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((X>1.01*self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((X<0.99*self.upper).double())==half_len
        bound = count_less==half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))
        
        return(torch.where(count_equal==0, median_out_cons, median_in_cons), self.w)

    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

dummy output:  tensor(5.6000, dtype=torch.float64)


In [15]:
# prover calculates settings, send to verifier

theory_output = torch.tensor(np.median(data))
lower_to_median = torch.tensor(np.sort(data)[int(len(data)/2)-1])
upper_to_median = torch.tensor(np.sort(data)[int(len(data)/2)])
print("Theory_output: ", theory_output)
print("lower: ", lower_to_median)
print("upper: ", upper_to_median)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = upper_to_median, requires_grad = False)
    def forward(self,X):
        # since within 1%, we regard as same value
        count_less = torch.sum((X < 0.99*self.w).double())
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        len = X.size()[1]
        half_len = torch.floor(torch.div(len, 2))
        
        # not support modulo yet
        less_cons = count_less<half_len+2*(len/2 - torch.floor(len/2))
        more_cons = count_less+count_equal>half_len

        # For count_equal == 0
        lower_exist = torch.sum((torch.abs(X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((X>1.01*self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((X<0.99*self.upper).double())==half_len
        bound = count_less==half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))
        
        return(torch.where(count_equal==0, median_out_cons, median_in_cons), self.w)


 

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

Theory_output:  tensor(49.5500, dtype=torch.float64)
lower:  tensor(49.3000, dtype=torch.float64)
upper:  tensor(49.8000, dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-25112,24986],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":12046,"total_const_size":1816,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [16]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 6.663752794265747 seconds
Theory output:  tensor(49.5500, dtype=torch.float64)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.55078125
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18341455175509539295, 12796101019039945164, 1607286914885633240, 1929881192315725821]]], 'proof': '0317ead9cc2f4dc243056ce8b8dace83fb768ec931f1fce5edefb45c41aacc942e326f1f834dc0d3d9b1008bb39677ca9abd990b3e83ec6a3ef8363aa9c3ac4a2f2e457367ed4e766682797cb65b7a4b0d1bf26a1348ca24cde7db501dbde98117230db2cf22b1e917a5224558dd42387c6eebc89b023e3b6a45b0b7d68c9d9406ade795340d29cc4d42489838963f089f73a971189576b43d382a8ac04b6d7508a7933b7dcadc4c8f2b875bbf61d95ea94504a10d79fa7c7466ecee00b36f98079203ee2824158a9c365f608913a311e56fed041775bfe21025f6469494f3f026079cf955507a10dc367a1a0ad7c4779721d32ac5494caf6b1b23755014d53410defe712f81a240288f90fbd420e7532f2afb8d6f4015ac7553ac23269cd0e506731432e7dfa485072b1ecd4848c527fefcb983073d98e5a75e3802d36c0dbd1ac61874d8ce26b425b41b88dbb0ace626a8b091d4274afa7bf317dadb72172b

In [17]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18341455175509539295, 12796101019039945164, 1607286914885633240, 1929881192315725821]]]
proof boolean:  1.0
proof result 1 : 49.55078125
verified
